In [1]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread 
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button( description="record", disabled=False, button_style="success", icon="mircophone")
stop_button = widgets.Button( description="stop", disabled=False, button_style="warning", icon="stop")

output = widgets.Output()

def start_recording(data):
    messages.put(True)

    with output:
        display("starting...")
        record = Thread(target = record_microphone)
        record.start()

        transcribe = Thread(target = speech_recognition , args=(output,))
        transcribe.start()

        
def stop_recording(data):
    with output:
        messages.get()
        display("stoppped.")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)


display(record_button, stop_button, output)



Button(button_style='success', description='record', icon='mircophone', style=ButtonStyle())

Button(button_style='warning', description='stop', icon='stop', style=ButtonStyle())

Output()

In [2]:
import pyaudio

p=pyaudio.PyAudio()

#for i in range(p.get_device_count()):
   # print(p.get_device_info_by_index(i))

p.terminate()


In [3]:
CHANNELS = 1
fr = 16000
rs = 20
af = pyaudio.paInt16
ss = 2

def record_microphone(chunk = 1024):
    p=pyaudio.PyAudio()

    stream = p.open(format = af , channels = CHANNELS , rate = fr , input = True , input_device_index = 1 ,frames_per_buffer=chunk)
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= fr*rs/chunk:
            recordings.put(frames.copy())

            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [4]:
import subprocess
import json
from vosk import Model , KaldiRecognizer

model = Model(model_name = "vosk-model-en-us-0.22")
rec = KaldiRecognizer(model , fr)
rec.SetWords(True)

def speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()
        rec.AcceptWaveform(b''.join(frames))

        result = rec.Result()
        text = json.loads(result)["text"]

        recase_script = r"C:\Users\shshv\voice_test\recase\recasepunc.py"
        checkpoint = r"C:\Users\shshv\voice_test\recase\checkpoint"

        try:
            cased = subprocess.check_output(
            ["python", recase_script, "predict", checkpoint],

            shell=True, text=True, input=text, stderr=subprocess.STDOUT
            )
            output.append_stdout(cased)
        except subprocess.CalledProcessError as e:
            output.append_stdout(f"Error: {e.output}")
